In [ ]:
import numpy as np
import pandas as pd
from power_flow import power_flow

# Definisco il modello
model=pyo.ConcreteModel()

# Variables gridIn/Out
model.pGridIn=pyo.Var(range(nTimeStamp),within=pyo.NonNegativeReals) #Definisce una variabile continua non negativa per ogni timestamp (nTimeStamp) che rappresenta l'energia prelevata dalla rete.
model.pGridOut=pyo.Var(range(nTimeStamp),within=pyo.NonNegativeReals) #Definisce una variabile continua non negativa per ogni timestamp che rappresenta l'energia immessa nella rete.

# Variables EV
# Questa parte del codice si definiscono delle variabili di ottimizzazione aggiuntive per il modello Pyomo. 
# In particolare, queste variabili rappresentano l'energia trasferita dalla rete ai veicoli elettrici 
# (pG2V - Grid to Vehicle) per ciascun veicolo e per ciascun intervallo di tempo. Ogni variabile ha un limite 
# massimo definito da PchMax_vehicles, che rappresenta la massima potenza di carica per ciascun veicolo.
for id,itm in enumerate([f'pG2V_car{idx}' for idx in range(nWB)]):
    model.add_component(itm,pyo.Var(range(nTimeStamp),within=pyo.NonNegativeReals,bounds=(0,PchMax_vehicles[id])))# lim max può essere definito come: PchMax_vehicles[id]

for id,itm in enumerate([f'pV2G_car{idx}' for idx in range(nWB)]):
    model.add_component(itm,pyo.Var(range(nTimeStamp),within=pyo.NonNegativeReals,bounds=(0,PdchMax_vehicles[id])))

for itm in [f'SOC_car{idx}' for idx in range(nWB)]:
    model.add_component(itm,pyo.Var(range(nTimeStamp),within=pyo.NonNegativeReals))

for itm in [f'voltage_WB{idx}' for idx in range(nWB)]:
    model.add_component(itm,pyo.Var(range(nTimeStamp),within=pyo.NonNegativeReals))

################################# Define objective function #####################################################################
def cost_function(model):
    cost_grid_in = sum(model.pGridIn[idx] * cost_per_kwh_grid_in[idx] for idx in range(nTimeStamp))
    cost_grid_out = sum(model.pGridOut[idx] * cost_per_kwh_grid_out[idx] for idx in range(nTimeStamp))
    
    # Calcolo del costo degli utenti per ogni timestamp
    costo_utenti_timestamp = []
    for idx in range(nTimeStamp):  
        costo_j = 0  # Inizializza il costo per questo timestamp
        for id in range(nWB):
            var_name = f'pG2V_car{id}'
            var_component = getattr(model, var_name)
            costo_j += var_component[idx] * user_cost[id]  # Accumula il costo per questo veicolo e timestamp
        costo_utenti_timestamp.append(costo_j)  # Aggiungi il costo totale di questo timestamp alla lista
    
    return cost_grid_in - cost_grid_out + sum(costo_utenti_timestamp)

model.obj=pyo.Objective(rule=cost_function, sense=pyo.minimize)

################################# Define equality constraints #####################################################################
# Assicura che la somma delle potenze prelevate, immesse, generate e consumate/dissipate sia bilanciata in ogni momento,
#  garantendo che l'energia sia conservata secondo il principio di conservazione dell'energia.
# A partire dall'equazioni di bilancio, si sono scritti tutti i flussi di potenza che in generale possono essere bidirezionali come
# un termine in ingresso e un termine in uscita, in modo che questi siano sempre valori positivi.

model.powerBalance = pyo.ConstraintList()
for i in range(nTimeStamp):
    exprex=power_consumption[i]-model.pGridIn[i] + model.pGridOut[i]- power_pv[i]
    for itm in range(nWB): 
        exprex+=model.__dict__[f'pG2V_car{itm}'][i]-model.__dict__[f'pV2G_car{itm}'][i]  #aggiungo al bilancio
        # anche le potenza prelevate e immesse da ogni veicolo elettrico
    model.powerBalance.add(expr=exprex==0)

################################# Define inequality constraints #####################################################################
        # constraint sul SOC minimo e massimo per ogni veicolo
            
model.minSOCconstr = pyo.ConstraintList()
model.maxSOCconstr = pyo.ConstraintList()
for i in range(nTimeStamp):
    for itm in range(nWB):
        model.minSOCconstr.add(model.__dict__[f'SOC_car{itm}'][i]>=SOC_min_vehicles[itm][i]*capacity_vehicles[itm])


for i in range(nTimeStamp):
    for itm in range(nWB):
        model.maxSOCconstr.add(model.__dict__[f'SOC_car{itm}'][i]<=SOC_max_vehicles[itm][i]*capacity_vehicles[itm])

# calcolo del SOC ad ogni passo temporale
def SOCcalc(model,i,itm):
    if i!=0:
        return model.__dict__[f'SOC_car{itm}'][i]==model.__dict__[f'SOC_car{itm}'][i-1]+(timeStampCars*(model.__dict__[f'pG2V_car{itm}'][i]*etaG2V-model.__dict__[f'pV2G_car{itm}'][i]/etaV2G))
    else:
        return  model.__dict__[f'SOC_car{itm}'][i]==SOC_0_vehicles[itm]*capacity_vehicles[itm]+(timeStampCars*(model.__dict__[f'pG2V_car{itm}'][i]*etaG2V-model.__dict__[f'pV2G_car{itm}'][i]/etaV2G))

model.SOCcalc= pyo.ConstraintList()
for i in range(nTimeStamp):
    for itm in range(nWB):
            model.SOCcalc.add(SOCcalc(model,i,itm))

    #constraint sulle tensioni minime e massime per ogni wallbox

model.minVoltageConstraint = pyo.ConstraintList()
model.maxVoltageConstraint = pyo.ConstraintList()

for idx in range(nWB):
    for t in range(nTimeStamp):
        # Supponiamo che min_voltage e max_voltage siano dizionari che forniscono i limiti di tensione per ogni wallbox e timestamp
        model.minVoltageConstraint.add(model.__dict__[f'voltage_WB{idx}'][t] >= min_voltage[idx][t])
        model.maxVoltageConstraint.add(model.__dict__[f'voltage_WB{idx}'][t] <= max_voltage[idx][t])

 # Aggiungo i vincoli che collegano le variabili di potenza alle variabili di tensione
 
model.voltageConstraint = pyo.ConstraintList()

for t in range(nTimeStamp):
    power_G2V_vector = [model.__dict__[f'pG2V_car{idx}'][t] for idx in range(nWB)]
    power_V2G_vector = [model.__dict__[f'pV2G_car{idx}'][t] for idx in range(nWB)]
    power_pv_t = power_pv[t]
    power_consumption_t = power_consumption[t]
    
    voltage_vector = power_flow(
        power_G2V_vector, power_V2G_vector, power_pv_t, power_consumption_t
    )
    
    for idx, voltage in enumerate(voltage_vector):
        model.voltageConstraint.add(
            model.__dict__[f'voltage_WB{idx}'][t] == voltage
        )

######################## Constraint sulle potenze massime allocabili per le colonnine in e out##################

# Definizione delle liste di vincoli
model.maxG2V = pyo.ConstraintList()
model.maxV2G = pyo.ConstraintList()

# Aggiungere i vincoli basati sullo schedule per cui potenza allocabile (per scarica o carica dell'EV) dovrà
#essere fissata a zero nei timestamp dove NON c'è una connessione di veicolo alla colonnina 

for i in range(nWB):
    for j in range(nTimeStamp):
        if schedule[i][j] == 0:
            # Se il veicolo non è connesso, la potenza deve essere zero
            model.maxG2V.add(model.__dict__[f'pG2V_car{i}'][j] == 0)
            model.maxV2G.add(model.__dict__[f'pV2G_car{i}'][j] == 0)
        else:
            # Se il veicolo è connesso, aggiungi i vincoli di potenza massima
            model.maxG2V.add(model.__dict__[f'pG2V_car{i}'][j] <= PchMax_vehicles[i])
            model.maxV2G.add(model.__dict__[f'pV2G_car{i}'][j] <= PdchMax_vehicles[i])

##########La seguente sezione è dedicata a rendere le variabile di grandezze bidirezionali mutualmente escludentesi,
# per cui un valore diverso da zero di una deve per forza annullare l'altro (non posso essere presenti contemporaneamente)############

######################## Constraint della rete in e out ###############

model.flag_dch_grid=pyo.Var(range(nTimeStamp), within=pyo.Binary)
model.flag_ch_grid=pyo.Var(range(nTimeStamp), within=pyo.Binary)

model.binaryGrid= pyo.ConstraintList()
for i in range(nTimeStamp):
    model.binaryGrid.add(expr=model.flag_ch_grid[i]+model.flag_dch_grid[i]<=1) 
#in questo modo si è fatto in modo di definire delle flag che fanno in modo che la grid_in e grid_out non siano
#contemporaneamente pari 1 (quindi che si preleva e immetta potenza dalla rete contemporaneamente)
model.maxGrid= pyo.ConstraintList()
for i in range(nTimeStamp):
    model.maxGrid.add(expr=model.pGridIn[i]<=model.flag_ch_grid[i]*pGridInMax)
    model.maxGrid.add(expr=model.pGridOut[i]<=model.flag_dch_grid[i]*pGridOutMax)


########################################## SOLVER #################################
# solver = pyo.SolverFactory('gurobi_direct')
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print("Optimal solution found")

    # Create a DataFrame for the resultes 
    results_data = []
    for h in range(nTimeStamp):
        results_data.append({
            "Time Step": h,
            'Power PV':power_pv[h],
            'P load':power_consumption[h],
            "pGridIn [kW]": model.pGridIn[h].value,
            "pGridOut [kW]": model.pGridOut[h].value,
            "pStorageDch [kW]":model.pStorageDch[h].value,
            "pStorageCh [kW]":model.pStorageCh[h].value,
            "SOC Storage [%]":(model.SOC_storage[h].value/storageCapacity)*100,
        })
        for itm in range(nWB):
            results_data[-1][f"pG2V_car{itm}"+' [kW]']=model.__dict__[f'pG2V_car{itm}'][h].value
            results_data[-1][f"pV2G_car{itm}"+' [kW]']=model.__dict__[f'pV2G_car{itm}'][h].value
            results_data[-1][f"SOC_car{itm}"+'%']=(model.__dict__[f'SOC_car{itm}'][h].value/capacity_vehicles[itm])*100

    results_df = pd.DataFrame(results_data)
results_df=results_df.rename({"Time Step":'Time Step '+str(timeStamp)+'h'},axis=1).set_index(f'Time Step {str(timeStamp)}'+'h')
results_df['Prices_grid_in [€/kWh]']=cost_per_kwh_grid_in
results_df['Prices_grid_out [€/kWh]']=cost_per_kwh_grid_out
results_df['Cost_grid_in [€]']=results_df.apply(lambda x: x['pGridIn [kW]']*x['Prices_grid_in [€/kWh]'],axis=1)
results_df['Cost_grid_out [€]']=results_df.apply(lambda x: x['pGridOut [kW]']*x['Prices_grid_out [€/kWh]'],axis=1)
results_df=results_df.round(2)


results_data = []
h=0
results_data.append({

    "pGridIn [kW]": model.pGridIn[h].value,
    "pGridOut [kW]": model.pGridOut[h].value,
    "pStorageDch [kW]":model.pStorageDch[h].value,
    "pStorageCh [kW]":model.pStorageCh[h].value,

})
for itm in range(nWB):
    results_data[-1][f"pG2V_car{itm}"+' [kW]']=model.__dict__[f'pG2V_car{itm}'][h].value
    results_data[-1][f"pV2G_car{itm}"+' [kW]']=model.__dict__[f'pV2G_car{itm}'][h].value




list(results_data[0].values()) # ritorno il valore 0 dei risultati che corrisonde al primo time step cioè quello da impostare dal sistema
